In [2]:
import math
import torch
import torch.nn as nn

In [16]:
d_model = 4
dropout = 0.5
batch_first = True
max_seq_len = 3

dropout = nn.Dropout(p=dropout)


# adapted from PyTorch tutorial
position = torch.arange(max_seq_len).unsqueeze(1)

div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))

if batch_first:
    pe = torch.zeros(1, max_seq_len, d_model)

    pe[0, :, 0::2] = torch.sin(position * div_term)

    pe[0, :, 1::2] = torch.cos(position * div_term)
else:
    pe = torch.zeros(max_seq_len, 1, d_model)

    pe[:, 0, 0::2] = torch.sin(position * div_term)

    pe[:, 0, 1::2] = torch.cos(position * div_term)

In [14]:
position

tensor([[0],
        [1],
        [2]])

In [17]:
div_term

tensor([1.0000, 0.0100])

In [18]:
position * div_term

tensor([[0.0000, 0.0000],
        [1.0000, 0.0100],
        [2.0000, 0.0200]])